In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import time
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
def create_cf_matrix(train, cf_type='user'):

    """
    Создает матрицу для коллаборативной фильтрации.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - Матрица для коллаборативной фильтрации.
    """
    if cf_type == 'user':
        # Матрица пользователь-фильм
        return train.pivot(index='userId', columns='movieId', values='rating')
    elif cf_type == 'item':
        # Матрица фильм-пользователь
        return train.pivot(index='movieId', columns='userId', values='rating')
    else:
        raise ValueError("cf_type должен быть 'user' или 'item'")
    
# Функция для замера времени
def time_function(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    elapsed_time = time.time() - start_time
    return result, elapsed_time

# Оформление результата
def create_results_dataframe(results):
    """
    Создает pandas DataFrame из словаря результатов.
    
    :param results: Словарь с названиями методов в качестве ключей и RMSE в качестве значений
    :return: pandas DataFrame с результатами
    """
    df = pd.DataFrame(results).T
    df.columns = ['RMSE', 'Время (с)']
    df.index.name = 'Метод'
    return df

# Загрузка и разделение данных
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    df = pd.read_csv(file_path)
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)
    return train, test

In [3]:
def svd():
    pass

In [4]:
def svd_p():
    pass

In [30]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Загрузка данных
df = pd.read_csv('data/ml-latest-small/ratings.csv')
n_users = df['userId'].nunique()
n_items = df['movieId'].nunique()

# Создание индексов для пользователей и фильмов
user_ids = df['userId'].astype('category').cat.codes.values
item_ids = df['movieId'].astype('category').cat.codes.values

df['user_idx'] = user_ids
df['item_idx'] = item_ids

# Разделение данных на тренировочные и тестовые наборы
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [19]:
# Матрица рейтингов
def create_matrix(df, n_users, n_items):
    matrix = np.zeros((n_users, n_items))
    for row in df.itertuples():
        matrix[row.user_idx, row.item_idx] = row.rating
    return matrix

R_train = create_matrix(train_df, n_users, n_items)
R_test = create_matrix(test_df, n_users, n_items)

In [37]:
# Реализация SVD с градиентным спуском
class SVD:
    def __init__(self, R, K=20, alpha=0.002, beta=0.02, iterations=100):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha  # Скорость обучения
        self.beta = beta  # Регуляризация
        self.iterations = iterations

    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R > 0])

        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        progress_bar = tqdm(total=self.iterations, desc="Training Progress")
        for iteration in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            progress_bar.set_postfix({'RMSE': f"{rmse:.4f}"})
            progress_bar.update(1)
        progress_bar.close()

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.predict(i, j)
            error = r - prediction

            # Обновление биасов
            self.b_u[i] += self.alpha * (error - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (error - self.beta * self.b_i[j])

            # Обновление скрытых факторов
            self.P[i, :] += self.alpha * (error * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (error * self.P[i, :] - self.beta * self.Q[j, :])

    def predict(self, i, j):
        return self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)

    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = []
        actual = []
        for x, y in zip(xs, ys):
            predicted.append(self.predict(x, y))
            actual.append(self.R[x, y])
        return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))

    def predict(self, i, j):
        return self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)

    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = []
        actual = []
        for x, y in zip(xs, ys):
            predicted.append(self.predict(x, y))
            actual.append(self.R[x, y])
        return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))

In [38]:
# Запуск модели
svd = SVD(R_train, K=20, iterations=50, alpha=0.002, beta=0.02)
svd.train()

Training Progress: 100%|██████████| 50/50 [01:05<00:00,  1.31s/it, RMSE=0.8091]


In [39]:
# Функция для расчета RMSE на тестовом наборе
def calculate_rmse(R_test, svd):
    xs, ys = R_test.nonzero()
    predicted = []
    actual = []
    progress_bar = tqdm(total=len(xs), desc="Calculating RMSE")
    for x, y in zip(xs, ys):
        predicted.append(svd.predict(x, y))
        actual.append(R_test[x, y])
        progress_bar.update(1)
    progress_bar.close()
    return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))

In [40]:
rmse_test = calculate_rmse(R_test, svd)
print(f"Test RMSE: {rmse_test:.4f}")

Calculating RMSE: 100%|██████████| 30251/30251 [00:00<00:00, 205793.02it/s]

Test RMSE: 0.8761


In [41]:
# Функция для расчета MAP@50
def calculate_map_k(R_test, svd, k=50):
    map_total = 0
    num_users = R_test.shape[0]

    progress_bar = tqdm(total=num_users, desc="Calculating MAP@50")
    for user in range(num_users):
        relevant_items = np.where(R_test[user, :] > 0)[0]
        if len(relevant_items) == 0:
            progress_bar.update(1)
            continue

        # Предсказания для всех фильмов
        predictions = [svd.predict(user, item) for item in range(R_test.shape[1])]
        top_k_items = np.argsort(predictions)[::-1][:k]

        # Вычисляем точность
        num_relevant = 0
        avg_precision = 0
        for i, item in enumerate(top_k_items):
            if item in relevant_items:
                num_relevant += 1
                avg_precision += num_relevant / (i + 1)

        if num_relevant > 0:
            map_total += avg_precision / min(len(relevant_items), k)

        progress_bar.update(1)
    
    progress_bar.close()
    return map_total / num_users

map_50 = calculate_map_k(R_test, svd, k=50)
print(f"MAP@50: {map_50:.4f}")

Calculating MAP@50: 100%|██████████| 610/610 [00:13<00:00, 44.40it/s]

MAP@50: 0.0197
